In [18]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, StackingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, mean_squared_error, accuracy_score, confusion_matrix, mean_absolute_error, r2_score
import xgboost as xgb
from xgboost import XGBClassifier, XGBRegressor
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.preprocessing import LabelEncoder
from scipy.stats import zscore
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import chi2, SelectFromModel
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.neural_network import MLPRegressor
from sklearn.cluster import KMeans
import itertools
from gurobipy import Model, GRB, quicksum

## Pre-Process

Load the Dataset

In [19]:
data = pd.read_csv('2021_Green_Taxi_Trip_Data.csv')

C:\Users\BK-FENS-1\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Remove Unnecessary Columns

In [20]:
data = data.drop(columns=['ehail_fee', 'improvement_surcharge', 'store_and_fwd_flag', 'RatecodeID', 'congestion_surcharge', 'fare_amount', 'extra', 'mta_tax'])

Check number of nulls

In [21]:
data.isnull().sum()

VendorID                 249115
lpep_pickup_datetime          0
lpep_dropoff_datetime         0
PULocationID                  0
DOLocationID                  0
passenger_count          412434
trip_distance                 0
tip_amount                    0
tolls_amount                  0
total_amount                  0
payment_type             412434
trip_type                412434
dtype: int64

Check payment types when tip amount > 0

In [22]:
filtered_data = data[data['tip_amount'] > 0]
filtered_data.groupby('payment_type').size()

payment_type
1.0    298760
3.0         1
dtype: int64

So we consider payment type 1.0 to be a credit card. If a tip was given, assume payment was made by credit card(1.0). (We have just one row with different payment type (3.0), so we change all of the rows with the tip into payment type = 1.0)

In [23]:
data.loc[(data['tip_amount'] > 0), 'payment_type'] = 1.0

Check trip types when passenger count > 4

In [24]:
filtered_data = data[data['passenger_count'] > 4]
filtered_data.groupby('trip_type').size()

trip_type
1.0    22695
2.0      382
dtype: int64

becuase we have more than 1 percent of both types, we add new type = 3.0 which means dispatch

Check Payment Types

In [25]:
data.groupby('payment_type').size()

payment_type
1.0    461268
2.0    259274
3.0      3891
4.0      1091
5.0         7
dtype: int64

number of payment type = 5.0 is 7, which is so little, we could consider payment type = 5.0 as unknown.

Classify Vans and Set Trip Type

In [26]:
data.loc[data['passenger_count'] > 4, 'trip_type'] = 3.0
data.loc[data['passenger_count'] > 4, 'payment_type'] = 5.0

Filter by Year (2021 Only)

In [27]:
data['pickup_datetime'] = pd.to_datetime(data['lpep_pickup_datetime'], format='%m/%d/%Y %I:%M:%S %p')
data['dropoff_datetime'] = pd.to_datetime(data['lpep_dropoff_datetime'], format='%m/%d/%Y %I:%M:%S %p')
data['duration'] = (data['dropoff_datetime'] - data['pickup_datetime']).dt.total_seconds() / 60
data = data[data['pickup_datetime'].dt.year == 2021]
data = data.drop(columns=['lpep_pickup_datetime','lpep_dropoff_datetime'])

In [28]:
data.isnull().mean()

VendorID            0.233096
PULocationID        0.000000
DOLocationID        0.000000
passenger_count     0.385912
trip_distance       0.000000
tip_amount          0.000000
tolls_amount        0.000000
total_amount        0.000000
payment_type        0.321153
trip_type           0.385912
pickup_datetime     0.000000
dropoff_datetime    0.000000
duration            0.000000
dtype: float64

Fill empty cells with RandomForest

In [29]:
def select_features(data, target_column, n_features=4):
    columns=[target_column]

    for column in data.columns:
        if(int(data[column].isnull().sum()) != 0 and column not in columns):
            columns.append(column)

    x_pass = data.drop(columns=columns)
    x_pass = x_pass.select_dtypes(include=[np.number])  # Keep only numeric columns
    y_pass = data[target_column].dropna()  
    x_pass = x_pass.loc[y_pass.index]  # Match indices of x_pass and y_pass

    # Train model
    model = RandomForestClassifier(n_estimators=50, random_state=0)
    model.fit(x_pass, y_pass)

    # Get feature importances
    feature_importances = pd.Series(model.feature_importances_, index=x_pass.columns)

    # Select top features
    selected_features = feature_importances.nlargest(n_features).index.tolist()
    return selected_features

def predict_with_selected_features(data , target_column):
    features = select_features(data,target_column)

    missing = data[data[target_column].isnull()]
    train_data = data.dropna(subset=[target_column])

    x_train = train_data[features]
    y_train = train_data[target_column]
    randomforest = RandomForestClassifier(n_estimators=50, random_state=0)
    randomforest.fit(x_train, y_train)

    data.loc[data[target_column].isnull(), target_column] = randomforest.predict(missing[features])
    return data
    

In [30]:
number_of_rows = data.shape[0]
data = data.dropna(subset=['VendorID'])

for column in data.columns:
    if(int(data[column].isnull().sum()) != 0):
            predict_with_selected_features(data,column)

data.isnull().sum()

VendorID            0
PULocationID        0
DOLocationID        0
passenger_count     0
trip_distance       0
tip_amount          0
tolls_amount        0
total_amount        0
payment_type        0
trip_type           0
pickup_datetime     0
dropoff_datetime    0
duration            0
dtype: int64

## First Part

Load & Add Boroughs

In [31]:
boroughs = pd.read_csv('Boroughs.csv')

In [32]:
data = data.merge(boroughs, left_on='PULocationID', right_on='LocationID', how='left')
data.rename(columns={'Borough': 'Origin'}, inplace=True)
data = data.merge(boroughs, left_on='DOLocationID', right_on='LocationID', how='left')
data.rename(columns={'Borough': 'Destination'}, inplace=True)

In [33]:
data

,VendorID,PULocationID,DOLocationID,passenger_count,trip_distance,tip_amount,tolls_amount,total_amount,payment_type,trip_type,pickup_datetime,dropoff_datetime,duration,LocationID_x,Origin,LocationID_y,Destination
0,2.0,43,151,1.0,1.01,0.00,0.0,6.80,2.0,1.0,2021-01-01 00:15:56,2021-01-01 00:19:52,3.933333,43,Manhattan,151,Manhattan
1,2.0,166,239,1.0,2.53,2.81,0.0,16.86,1.0,1.0,2021-01-01 00:25:59,2021-01-01 00:34:44,8.750000,166,Manhattan,239,Manhattan
2,2.0,41,42,1.0,1.12,1.00,0.0,8.30,1.0,1.0,2021-01-01 00:45:57,2021-01-01 00:51:55,5.966667,41,Manhattan,42,Manhattan
3,2.0,265,265,3.0,0.00,0.00,0.0,-52.80,3.0,1.0,2021-01-01 00:16:36,2021-01-01 00:16:40,0.066667,265,Manhattan,265,Manhattan
4,2.0,265,265,3.0,0.00,0.00,0.0,52.80,2.0,1.0,2021-01-01 00:16:36,2021-01-01 00:16:40,0.066667,265,Manhattan,265,Manhattan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
819605,2.0,77,93,1.0,10.81,0.00,0.0,49.20,2.0,1.0,2021-12-31 23:54:00,2022-01-01 00:19:00,25.000000,77,Brooklyn,93,Queens
819606,2.0,189,4,1.0,4.65,3.00,0.0,28.39,1.0,1.0,2021-12-31 23:37:00,2021-12-31 23:56:00,19.000000,189,Brooklyn,4,Manhattan
819607,2.0,41,137,1.0,6.70,6.00,0.0,33.23,1.0,1.0,2021-12-31 23:59:00,2022-01-01 00:16:00,17.000000,41,Manhattan,137,Manhattan
819608,2.0,97,262,1.0,10.38,8.42,0.0,46.55,1.0,1.0,2021-12-31 23:08:00,2021-12-31 23:29:00,21.000000,97,Brooklyn,262,Manhattan


Make OD matrix

In [34]:
od_matrix = data.groupby(['Origin','Destination']).size().unstack(fill_value=0)
od_matrix

Destination,Bronx,Brooklyn,EWR,Manhattan,Queens,Staten Island
Origin,,,,,,
Bronx,40329,4926,6,17613,4637,408
Brooklyn,4820,116100,227,24421,14280,1282
EWR,0,0,11,7,0,0
Manhattan,30497,8041,180,360777,12236,336
Queens,4820,13619,21,19639,137736,374
Staten Island,342,869,5,316,381,354


Trip Percent

In [35]:
total_trips = sum(od_matrix.sum())
od_matrix_percent = (od_matrix / total_trips) * 100
od_matrix_percent

Destination,Bronx,Brooklyn,EWR,Manhattan,Queens,Staten Island
Origin,,,,,,
Bronx,4.920511,0.601018,0.000732,2.148949,0.565757,0.049780
Brooklyn,0.588085,14.165274,0.027696,2.979588,1.742292,0.156416
EWR,0.000000,0.000000,0.001342,0.000854,0.000000,0.000000
Manhattan,3.720916,0.981076,0.021962,44.018131,1.492905,0.040995
Queens,0.588085,1.661644,0.002562,2.396140,16.805066,0.045631
Staten Island,0.041727,0.106026,0.000610,0.038555,0.046486,0.043191


In [36]:
od_flat = od_matrix_percent.stack().reset_index()
od_flat.columns = ['Origin', 'Destination', 'Percentage']
od_sorted = od_flat.sort_values(by='Percentage', ascending=False)
od_sorted

,Origin,Destination,Percentage
21,Manhattan,Manhattan,44.018131
28,Queens,Queens,16.805066
7,Brooklyn,Brooklyn,14.165274
0,Bronx,Bronx,4.920511
18,Manhattan,Bronx,3.720916
9,Brooklyn,Manhattan,2.979588
27,Queens,Manhattan,2.396140
3,Bronx,Manhattan,2.148949
10,Brooklyn,Queens,1.742292
25,Queens,Brooklyn,1.661644


Different Origin and Destination Percent Trip

In [37]:
for index, row in od_sorted.iterrows():
    if(row['Origin']!=row['Destination'] and row['Percentage']>=0.1):
        print('Origin:',row['Origin'],',','Destination:',row['Destination'],',','Percentage',row['Percentage'])

Origin: Manhattan , Destination: Bronx , Percentage 3.720916045436244
Origin: Brooklyn , Destination: Manhattan , Percentage 2.979587852759239
Origin: Queens , Destination: Manhattan , Percentage 2.3961396273837554
Origin: Bronx , Destination: Manhattan , Percentage 2.1489488903258867
Origin: Brooklyn , Destination: Queens , Percentage 1.7422920657385832
Origin: Queens , Destination: Brooklyn , Percentage 1.6616439526116078
Origin: Manhattan , Destination: Queens , Percentage 1.4929051622112954
Origin: Manhattan , Destination: Brooklyn , Percentage 0.9810763655885115
Origin: Bronx , Destination: Brooklyn , Percentage 0.6010175571308305
Origin: Queens , Destination: Bronx , Percentage 0.5880845768109223
Origin: Brooklyn , Destination: Bronx , Percentage 0.5880845768109223
Origin: Bronx , Destination: Queens , Percentage 0.5657568843718354
Origin: Brooklyn , Destination: Staten Island , Percentage 0.15641585632190919
Origin: Staten Island , Destination: Brooklyn , Percentage 0.1060260367

Same Origin and Destination Percent Trip

In [38]:
for index, row in od_sorted.iterrows():
    if(row['Origin']==row['Destination'] and row['Percentage']>=0.1):
        print('Origin:',row['Origin'],',','Destination:',row['Destination'],',','Percentage',row['Percentage'])

Origin: Manhattan , Destination: Manhattan , Percentage 44.01813057429753
Origin: Queens , Destination: Queens , Percentage 16.80506582398946
Origin: Brooklyn , Destination: Brooklyn , Percentage 14.165273727748565
Origin: Bronx , Destination: Bronx , Percentage 4.920510974731885


Balance OD matrix

In [39]:
balance_factor = number_of_rows / total_trips
od_matrix_balanced = od_matrix * balance_factor
od_matrix_balanced

Destination,Bronx,Brooklyn,EWR,Manhattan,Queens,Staten Island
Origin,,,,,,
Bronx,52586.730915,6423.224887,7.823660,22966.354028,6046.385263,532.008882
Brooklyn,6285.006894,151387.821647,295.995138,31843.600279,18620.310880,1671.655360
EWR,0.000000,0.000000,14.343377,9.127603,0.000000,0.000000
Manhattan,39766.360007,10485.008388,234.709801,470432.765980,15955.050695,438.124962
Queens,6285.006894,17758.404333,27.382810,25608.143233,179599.939727,487.674809
Staten Island,445.948622,1133.126762,6.519717,412.046095,496.802412,461.595942


## Second Part

Find Daily Demand of Trip from each zone to each zone and Balance

In [40]:
data['Trips']=1
data['Date'] = data['pickup_datetime'].dt.date
daily_trips = data.groupby(['PULocationID', 'DOLocationID', 'Date'])['Trips'].sum().reset_index()
balance_factor = number_of_rows / total_trips
daily_trips['Trips'] = daily_trips['Trips'] * balance_factor
average_daily_trips = daily_trips.groupby(['PULocationID', 'DOLocationID'])['Trips'].mean().reset_index()
average_daily_trips.rename(columns={'Trips': 'Average_Daily_Trips'}, inplace=True)

Select 25*25 Most weighted matrix

In [41]:
average_daily_trips_sorted = average_daily_trips.sort_values(by='Average_Daily_Trips', ascending=False)

od_matrix = average_daily_trips.pivot(
    index='PULocationID', columns='DOLocationID', values='Average_Daily_Trips'
).fillna(0)


row_sums = od_matrix.sum(axis=1)
col_sums = od_matrix.sum(axis=0)
total_trip = row_sums + col_sums

row_spread = (od_matrix > 0).sum(axis=1)
col_spread = (od_matrix > 0).sum(axis=0)
total_spread = row_spread + col_spread

location_scores = (total_trip * 0.7) + (total_spread * 0.3)

top_locations = location_scores.sort_values(ascending=False).head(25).index

trip_demand = od_matrix.loc[top_locations, top_locations]

trip_demand = trip_demand.fillna(0)

trip_demand


,74,75,41,42,244,95,166,76,7,61,...,55,188,14,39,25,213,35,197,89,226
74,29.708473,53.601002,26.568292,23.478125,7.880976,1.303943,20.266495,1.354095,1.732246,1.404247,...,1.303943,1.303943,1.303943,1.303943,1.385440,1.624755,1.303943,1.371975,1.369141,1.379171
75,55.540841,28.625855,17.433544,14.875671,3.637753,1.633676,7.819987,1.412605,1.691175,1.425240,...,1.459460,1.363213,1.303943,1.356101,1.434338,1.661320,1.303943,1.393870,1.385440,1.462366
41,21.052953,12.810797,22.915730,26.825508,3.985565,1.441201,10.876971,1.303943,1.470404,1.470404,...,1.303943,1.358274,1.564732,1.404247,1.412605,1.405814,1.303943,1.303943,1.303943,1.607186
42,10.186607,6.802544,15.238942,20.855929,4.522438,1.303943,4.690470,1.303943,1.448826,1.348907,...,1.422484,1.363213,1.564732,1.303943,1.303943,1.593709,1.380646,1.303943,1.404247,1.564732
244,2.819835,2.392453,2.674189,4.508816,8.028411,1.341199,5.149800,1.461997,1.303943,1.303943,...,1.303943,1.303943,1.380646,1.303943,1.303943,1.448826,1.303943,1.357166,1.422484,1.303943
95,1.303943,1.338258,1.303943,1.434338,1.360637,24.814328,1.390873,1.490221,1.754571,1.355078,...,1.303943,1.303943,1.303943,1.303943,1.303943,1.303943,1.376385,3.170953,1.303943,1.945781
166,7.849014,4.489526,8.192563,6.901717,5.094123,1.303943,10.935834,1.303943,1.388069,1.303943,...,1.303943,1.303943,1.303943,1.303943,1.303943,1.303943,1.303943,1.303943,1.303943,1.303943
76,1.382970,1.521267,1.422484,1.385440,1.504550,1.376385,1.629929,3.072718,1.303943,1.804447,...,1.405814,1.801501,1.380646,1.659564,1.412605,1.521267,1.833267,1.352238,1.452119,1.303943
7,1.493225,1.427289,1.378454,1.434338,1.408259,1.609372,1.372572,1.422484,14.606327,1.400532,...,1.412605,1.303943,1.303943,1.303943,1.303943,1.303943,1.303943,1.376385,1.303943,3.793673
61,1.303943,1.376385,1.352238,1.422484,1.303943,1.303943,1.303943,1.918659,1.303943,3.536327,...,1.399354,2.019009,1.420714,1.899711,1.664149,1.303943,1.652325,1.303943,1.535755,1.303943


Cluster day hours

In [42]:
data['hour'] = data['pickup_datetime'].dt.hour
data['day'] = data['pickup_datetime'].dt.weekday
hourly_trips = data.groupby('hour').size().reset_index(name='Trip_Count')

X = hourly_trips[['Trip_Count']]

kmeans = KMeans(n_clusters=4, random_state=42)
hourly_trips['Cluster'] = kmeans.fit_predict(X)

hour_to_cluster = dict(zip(hourly_trips['hour'], hourly_trips['Cluster']))

data['Time_Group'] = data['hour'].map(hour_to_cluster)

Calc cost matrix

In [43]:
filtered_data = data[data['PULocationID'].isin(top_locations) & data['DOLocationID'].isin(top_locations)]
average_costs = filtered_data.groupby(['PULocationID', 'DOLocationID', 'Time_Group','day'])['total_amount'].mean().reset_index()
average_costs.rename(columns={'total_amount': 'Amount'}, inplace=True)

print(average_costs)

all_combinations = pd.DataFrame(
    list(itertools.product(top_locations, top_locations, filtered_data['Time_Group'].unique(),filtered_data['day'].unique())),
    columns=['PULocationID', 'DOLocationID', 'Time_Group','day']
)

merged = all_combinations.merge(average_costs, on=['PULocationID', 'DOLocationID', 'Time_Group','day'], how='left', indicator=True)

missing_combinations = merged[merged['_merge'] == 'left_only'][['PULocationID', 'DOLocationID', 'Time_Group','day']]

print(missing_combinations)

      PULocationID  DOLocationID  Time_Group  day     Amount
0                7             7           0    0  11.746316
1                7             7           0    1  10.412778
2                7             7           0    2  11.813585
3                7             7           0    3  10.260943
4                7             7           0    4  13.130000
...            ...           ...         ...  ...        ...
8019           244           244           3    2   7.689687
8020           244           244           3    3  10.370000
8021           244           244           3    4  10.513250
8022           244           244           3    5   9.476585
8023           244           244           3    6  11.730769

[8024 rows x 5 columns]
       PULocationID  DOLocationID  Time_Group  day
140              74            95           2    4
141              74            95           2    5
142              74            95           2    6
143              74            95      

In [44]:
clusters = filtered_data.groupby(['Time_Group']).size()
balance_factor = number_of_rows / total_trips
clusters = clusters * balance_factor
cluters_percent = clusters/sum(clusters)
print(cluters_percent)
print()

days = filtered_data.groupby(['day']).size()
days = days * balance_factor
days_percent = days/sum(days)
print(days_percent)

Time_Group
0    0.094711
1    0.729034
2    0.041558
3    0.134696
dtype: float64

day
0    0.140195
1    0.149129
2    0.157174
3    0.158424
4    0.164526
5    0.133451
6    0.097102
dtype: float64


fill missing value with inf

In [45]:
average_costs = all_combinations.merge(average_costs, on=['PULocationID', 'DOLocationID', 'Time_Group','day'], how='left')
average_costs['Amount'] = average_costs['Amount'].fillna(float('inf'))

average_costs

,PULocationID,DOLocationID,Time_Group,day,Amount
0,74,74,2,4,9.724000
1,74,74,2,5,11.982708
2,74,74,2,6,12.081220
3,74,74,2,0,11.842963
4,74,74,2,1,9.549167
...,...,...,...,...,...
17495,226,226,1,6,17.497179
17496,226,226,1,0,14.005467
17497,226,226,1,1,15.210833
17498,226,226,1,2,12.334792


Find Distance between each locations and Fill NaN

In [46]:
average_dist = filtered_data.groupby(['PULocationID', 'DOLocationID'])['trip_distance'].mean().reset_index()
average_dist.rename(columns={'trip_distance': 'Dist'}, inplace=True)

all_combinations = pd.DataFrame(
    list(itertools.product(top_locations, top_locations)),
    columns=['PULocationID', 'DOLocationID']
)

average_dist = all_combinations.merge(average_dist, on=['PULocationID', 'DOLocationID'], how='left')
average_dist['Dist'] = average_dist['Dist'].fillna(float('inf'))

average_dist


,PULocationID,DOLocationID,Dist
0,74,74,7.791831
1,74,75,1.345465
2,74,41,16.018143
3,74,42,51.463034
4,74,244,3.505723
...,...,...,...
620,226,213,10.490000
621,226,35,9.852222
622,226,197,9.221500
623,226,89,1987.430000


Deterance function Exponential

In [47]:
def deterrence_function_expo(cost, alpha, beta):
    return alpha * np.exp(-beta * cost)

Deterance function Lognormal

In [48]:
def deterrence_function_ln(cost, alpha, beta):
    return alpha * np.exp(-beta * np.log(cost + 1)**2)

Gravity Model

In [49]:
def gravity_model(c_matrix, o_values, d_values, type, alpha=1, beta=0.1):
    
    sum_O = np.sum(o_values)
    sum_D = np.sum(d_values)
    
    if sum_O != sum_D:
        if sum_O < sum_D:
            correction_ratio = sum_D / sum_O
            o_values = o_values * correction_ratio
        else:
            correction_ratio = sum_O / sum_D
            d_values = d_values * correction_ratio
    
    if type == 'ln':
        deterrence = np.vectorize(deterrence_function_ln)(c_matrix, alpha, beta)
    elif type == 'expo':
        deterrence = np.vectorize(deterrence_function_expo)(c_matrix, alpha, beta)

    num_zones = len(o_values)
    A = np.ones(num_zones)
    B = np.ones(num_zones)

    
    error = 1
    threshold = 0.03
    n = 0

    while error > threshold:
        n+=1
        # Update A factors
        for i in range(num_zones):
            A[i] = 1 / (np.sum(B * d_values * deterrence[i, :]) + 1e-9)

        # Update B factors
        for j in range(num_zones):
            B[j] = 1 / (np.sum(A * o_values * deterrence[:, j]) + 1e-9)

        # Calculate Tij matrix for the model
        t_matrix = np.outer(A * o_values, B * d_values) * deterrence

        # Compute error
        error = (
            np.sum(np.abs(o_values - np.sum(t_matrix, axis=1)))
            + np.sum(np.abs(d_values - np.sum(t_matrix, axis=0)))
        ) / np.sum(o_values)

        #if couldnt reach better error
        if n>=1000000:
            print( 'error after 10^6 times error is:', error)
            print('Couldnt reach better error')
            break
    
    return t_matrix    



Calc Average Cost of all periods of all days

In [50]:
average_cost_alltimes = filtered_data.groupby(['PULocationID', 'DOLocationID'])['total_amount'].mean().reset_index()
average_cost_alltimes.rename(columns={'total_amount': 'Amount'}, inplace=True)

all_combinations = pd.DataFrame(
    list(itertools.product(top_locations, top_locations)),
    columns=['PULocationID', 'DOLocationID']
)

average_cost_alltimes = all_combinations.merge(average_cost_alltimes, on=['PULocationID', 'DOLocationID'], how='left')
average_cost_alltimes['Amount'] = average_cost_alltimes['Amount'].fillna(float('inf'))

average_cost_alltimes

,PULocationID,DOLocationID,Amount
0,74,74,8.321417
1,74,75,9.793480
2,74,41,8.674643
3,74,42,10.075269
4,74,244,18.184655
...,...,...,...
620,226,213,43.306250
621,226,35,35.201111
622,226,197,34.675000
623,226,89,42.682222


Run and Compare deterances

In [51]:
o = trip_demand.sum(axis=1).to_numpy()
d = trip_demand.sum(axis=0).to_numpy()

c_matrix = average_cost_alltimes.pivot_table(
    index='PULocationID',
    columns='DOLocationID',
    values='Amount',
    fill_value=np.inf
)

t_matrix_actual = trip_demand.to_numpy()

t_matrix_expo = gravity_model(c_matrix, o, d, type='expo')
t_matrix_ln = gravity_model(c_matrix, o, d, type='ln')

rmse_expo = np.sqrt(mean_squared_error(t_matrix_actual.flatten(), t_matrix_expo.flatten()))
rmse_ln = np.sqrt(mean_squared_error(t_matrix_actual.flatten(), t_matrix_ln.flatten()))

r2_expo = r2_score(t_matrix_actual.flatten(), t_matrix_expo.flatten())
r2_ln = r2_score(t_matrix_actual.flatten(), t_matrix_ln.flatten())

print(f"RMSE (Expo): {rmse_expo}, R^2 (Expo): {r2_expo}")
print(f"RMSE (LN): {rmse_ln}, R^2 (LN): {r2_ln}")


RMSE (Expo): 5.458092909503267, R^2 (Expo): -0.49034012223016266
RMSE (LN): 3.3389301833761857, R^2 (LN): 0.44227742802287373


The LN model clearly performs better than the Expo model, both in terms of error (RMSE) and the proportion of variance explained (R2).
The Expo model's R2 being negative indicates that it fails to capture the data's trend and performs poorly compared to the mean as a predictor.

Add Tip piad or not to dataset

In [52]:
data['tip_paid'] = data['tip_amount'].apply(lambda x: 1 if x > 0 else 0)
features=['PULocationID', 'DOLocationID', 'trip_distance', 'Time_Group', 'day', 'total_amount']
X = data[features]
y = data['tip_paid']  # Target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


Aggregate Data

In [53]:
merged_data = average_dist.merge(
    average_costs,
    on=['PULocationID', 'DOLocationID'],
    how='inner'
)

merged_data.rename(columns={'Amount': 'total_amount', 'Dist': 'trip_distance'}, inplace=True)
merged_data

,PULocationID,DOLocationID,trip_distance,Time_Group,day,total_amount
0,74,74,7.791831,2,4,9.724000
1,74,74,7.791831,2,5,11.982708
2,74,74,7.791831,2,6,12.081220
3,74,74,7.791831,2,0,11.842963
4,74,74,7.791831,2,1,9.549167
...,...,...,...,...,...,...
17495,226,226,1.391308,1,6,17.497179
17496,226,226,1.391308,1,0,14.005467
17497,226,226,1.391308,1,1,15.210833
17498,226,226,1.391308,1,2,12.334792


In [54]:
merged_data_for_prediction = merged_data.copy()
merged_data_for_prediction.replace([np.inf, -np.inf], np.finfo(np.float32).max, inplace=True)

Prediction Tip piad or not

In [55]:
# base models
base_models = [
    ('decision_tree', DecisionTreeClassifier(random_state=42)),
    ('xgboost', xgb.XGBClassifier(random_state=42))
]

# meta model
meta_model = LogisticRegression()

# stacking model
stacking_model = StackingClassifier(estimators=base_models, final_estimator=meta_model, cv=5)
stacking_model.fit(X_train, y_train)

y_pred = stacking_model.predict(X_test)
cm = confusion_matrix(y_test, y_pred, labels=[0, 1])
TN, FP, FN, TP = cm.ravel()
error_rate = (FP + FN) / (TP + TN + FP + FN)
print(error_rate)

merged_data['tip_paid'] = stacking_model.predict(merged_data_for_prediction)


0.12731055013970058


In [56]:
tip_prob=1-error_rate
print('tip prob:',tip_prob)
PULocationID=merged_data['PULocationID'].unique()
DOLocationID=merged_data['DOLocationID'].unique()
Time_Group=merged_data['Time_Group'].unique()
day=merged_data['day'].unique()
merged_data

tip prob: 0.8726894498602994


,PULocationID,DOLocationID,trip_distance,Time_Group,day,total_amount,tip_paid
0,74,74,7.791831,2,4,9.724000,1
1,74,74,7.791831,2,5,11.982708,0
2,74,74,7.791831,2,6,12.081220,0
3,74,74,7.791831,2,0,11.842963,0
4,74,74,7.791831,2,1,9.549167,1
...,...,...,...,...,...,...,...
17495,226,226,1.391308,1,6,17.497179,0
17496,226,226,1.391308,1,0,14.005467,0
17497,226,226,1.391308,1,1,15.210833,1
17498,226,226,1.391308,1,2,12.334792,0


## Optimization

Model

In [80]:
model = Model("Maximize_Revenue")

Variable

In [81]:
x = model.addVars(PULocationID, DOLocationID, Time_Group, day, vtype=GRB.INTEGER, name="x")

Parameters

In [82]:
distance = {}
for _, row in merged_data.iterrows():
    if row['trip_distance'] >= 1e10:
        distance[(row['PULocationID'], row['DOLocationID'])] = GRB.INFINITY
    else:
        distance[(row['PULocationID'], row['DOLocationID'])] = row['trip_distance']

total_amount = {}
for _, row in merged_data.iterrows():
    if row['total_amount'] >= 1e10:
        total_amount[(row['PULocationID'], row['DOLocationID'], row['Time_Group'], row['day'])] = 0
    else:
        total_amount[(row['PULocationID'], row['DOLocationID'], row['Time_Group'], row['day'])] = row['total_amount']

tip_paid = {}
for _, row in merged_data.iterrows():
    tip_paid[(row['PULocationID'], row['DOLocationID'], row['Time_Group'], row['day'])] = row['tip_paid']

working_days = [0, 1, 2, 3, 4]

Object

In [83]:
model.setObjective(
    sum(
        40 * x[i, j, t, d] + 5 * tip_prob * x[i, j, t, d] * tip_paid[i,j,t,d]
        for i in PULocationID for j in DOLocationID for t in Time_Group for d in day
    ),
    GRB.MAXIMIZE
)

Constraints

In [84]:
# Demand constraint
pulocation_map = {loc_id: idx for idx, loc_id in enumerate(PULocationID)}
dolocation_map = {loc_id: idx for idx, loc_id in enumerate(DOLocationID)}

for i in PULocationID:
    for j in DOLocationID:
        for t in Time_Group:
            for d in day:
                
                i_idx = pulocation_map.get(i, -1)
                j_idx = dolocation_map.get(j, -1)
                demand_value = t_matrix_ln[i_idx, j_idx]

                model.addConstr(
                    x[i, j, t, d] <= demand_value * days_percent[d] * cluters_percent[t] * 7,
                    name=f"DemandConstraint_{i}_{j}_{t}_{d}"
                )

In [85]:
# Maximum distance per day (1000 km per day)
for d in day:
    model.addConstr(
        quicksum(distance[i, j] * x[i, j, t, d] for i in PULocationID for j in DOLocationID for t in Time_Group) <= 1000,
        name=f"DistanceConstraint_{d}"
    )


In [86]:
# Working Days Constraint
for i in PULocationID:
    for j in DOLocationID:
        for t in Time_Group:
            for d in day:
                if d not in working_days:
                    model.addConstr(x[i, j, t, d] == 0, name=f"NonWorkingDays_{i}_{j}_{t}_{d}")

In [87]:
# Minimum Trip Cost (40$) Constraint
for i in PULocationID:
    for j in DOLocationID:
        for t in Time_Group:
            for d in day:
                if total_amount[i, j, t, d] < 40:
                    model.addConstr(x[i, j, t, d] == 0, name=f"MinCostConstraint_{i}_{j}_{t}_{d}")


Solve the code

In [88]:
model.optimize()

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Xeon(R) Gold 6248R CPU @ 3.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 48 physical cores, 96 logical processors, using up to 32 threads

Optimize a model with 37491 rows, 17500 columns and 54956 nonzeros
Model fingerprint: 0x9fa7d83a
Variable types: 0 continuous, 17500 integer (0 binary)
Coefficient statistics:
  Matrix range     [7e-01, 1e+100]
  Objective range  [4e+01, 4e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-02, 1e+03]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Found heuristic solution: objective 14591.262704
Presolve removed 37490 rows and 17352 columns
Presolve time: 0.13s
Presolved: 1 rows, 148 columns, 148 nonzeros
Found heuristic solution: objective 18967.966716
Variable types: 0 continuous, 148 integer (70 binary)
Found heuristic solution: objective 19184.691530

Root rela

Result

In [89]:
if model.status == GRB.OPTIMAL:
    print(f"Objective Value (Revenue): {model.objVal}")
    for v in model.getVars():
        if v.x > 1:
            print(f"{v.varName}: {v.x}")
else:
    print("No optimal solution found.")

Objective Value (Revenue): 19695.96463561938
x[74,95,1,4]: 5.0
x[74,95,1,1]: 5.0
x[74,95,1,2]: 5.0
x[74,95,1,3]: 5.0
x[74,82,1,3]: 4.0
x[74,130,1,0]: 3.0
x[74,97,1,4]: 6.0
x[74,97,1,0]: 5.0
x[74,97,1,1]: 6.0
x[74,97,1,2]: 4.0
x[74,97,1,3]: 5.0
x[74,65,1,4]: 2.0
x[74,65,1,1]: 2.0
x[74,65,1,2]: 2.0
x[74,65,1,3]: 2.0
x[74,25,1,4]: 4.0
x[74,25,1,1]: 3.0
x[74,25,1,2]: 3.0
x[74,197,1,0]: 3.0
x[74,89,1,0]: 5.0
x[74,89,1,3]: 4.0
x[75,95,1,4]: 4.0
x[75,95,1,0]: 3.0
x[75,95,1,1]: 3.0
x[75,95,1,2]: 3.0
x[75,95,1,3]: 4.0
x[75,61,1,0]: 3.0
x[75,61,1,1]: 2.0
x[75,65,1,2]: 4.0
x[75,197,1,0]: 2.0
x[41,95,1,4]: 2.0
x[41,95,1,1]: 2.0
x[41,95,1,2]: 2.0
x[41,61,1,4]: 2.0
x[41,130,1,0]: 2.0
x[41,97,1,0]: 2.0
x[41,97,1,3]: 2.0
x[41,65,1,4]: 3.0
x[41,65,1,0]: 2.0
x[41,65,1,1]: 2.0
x[41,65,1,2]: 2.0
x[41,65,1,3]: 2.0
x[42,95,1,4]: 2.0
x[42,95,1,1]: 2.0
x[42,7,1,2]: 3.0
x[42,97,1,1]: 2.0
x[42,97,1,2]: 2.0
x[42,97,1,3]: 2.0
x[42,65,1,4]: 2.0
x[42,65,1,0]: 2.0
x[42,65,1,1]: 2.0
x[42,65,1,2]: 2.0
x[42,65,1,3]: 2.

In [91]:
print("Revenue in a year:",model.objVal*52.1429)

Revenue in a year: 1027004.7143986378
